<a href="https://colab.research.google.com/github/jihoon99/bert_SQuAD/blob/master/3_Bert_squad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQuAD2.0 dataset
- rajpurkar.github.io/SQuAD-explorer/
- Stanford Question Answering Dataset
- 답변의 문장의 시작을 스코어링함(1)
- 답변의 문장의 끝을 스코어링함(2)

# installation

In [1]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 4.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-cp37-none-any.whl size=30535 sha256=f1f8e2f1a04d4eefdc48db58a1a13872d6fdf4fbec2f0e0ea8a02f2cff62f6a2
  Stored in directory: /root/.cache/pip/wheels/a1/04/ee/347bd9f5b821b637c76411d280271a857aece00358896a230f
  Created wheel for py-params: filename=py_params-0.10.2-cp37-none-any.whl size=7912 sha256=6dfa05ea4a6cf4655777cbaa3a82619cf6f60a4003628bccb1ed49c2cb9770ee
  Stored in directory: /root/.cache/pip/wheels/d0/4a/70/ff12450229ff1955abf01f365051d4faae1c20aef53ab4cf09
  Created wheel for params-flow: filename=params_flow-0.8.2-cp37-none-any.whl size=19472 sha256=666cdbff913b50911d16cef21deed5f3061eb9d9ba13d99b1cfe4f97d0a891f5
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.2MB 6.6MB/s 


In [2]:
# !pip install tf-models-nightly
# !pip install --upgrade tf-nightly-gpu
!pip install -q tf-models-official==2.4.0

     |████████████████████████████████| 1.1MB 5.5MB/s 
     |████████████████████████████████| 358kB 19.1MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 215kB 23.0MB/s 
     |████████████████████████████████| 686kB 20.8MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 37.2MB 80kB/s 
     |████████████████████████████████| 645kB 22.5MB/s 


In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import tensorflow_hub as hub
# where to download models
from tensorflow.keras import layers
import bert

In [4]:
# https://github.com/tensorflow/models/blob/master/official/nlp/data/squad_lib.py


import tensorflow_hub as hub

from official.nlp.bert.tokenization import FullTokenizer
from official.nlp.bert.input_pipeline import create_squad_dataset
from official.nlp import optimization

from official.nlp.data.squad_lib import generate_tf_record_from_json_file # squad dataset이 json임.
from official.nlp.data.squad_lib import read_squad_examples, FeatureWriter, convert_examples_to_features, write_predictions
# FeatureWriter : """Writes InputFeature to TF example file."""


import numpy as np
import math
import random
import time
import json
import collections
import os

from google.colab import drive

#1. Load Squad Dataset and rewrite as train_meta_data

In [5]:
drive.mount("/content/drive")

input_meta_data = generate_tf_record_from_json_file(
    "/content/drive/MyDrive/study/udemy_bert/data/train-v1.1.json",
    "/content/drive/MyDrive/study/udemy_bert/data/vocab.txt",
    "/content/drive/MyDrive/study/udemy_bert/data/train-v1.1.tf_record")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
with tf.io.gfile.GFile("/content/drive/MyDrive/study/udemy_bert/data/train_meta_data", "w") as writer:
    writer.write(json.dumps(input_meta_data, indent=4) + "\n")

In [7]:
BATCH_SIZE = 4 # 데이터셋 크기가 커서 배치를 4개씩..

train_dataset = create_squad_dataset(
    "/content/drive/MyDrive/study/udemy_bert/data/train-v1.1.tf_record",
    input_meta_data['max_seq_length'], # 384 # maximum length of sequence
    BATCH_SIZE,
    is_training=True)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# 2. modeling

In [8]:
class BertSquadLayer(tf.keras.layers.Layer):

  def __init__(self):
    super(BertSquadLayer, self).__init__()
    self.final_dense = tf.keras.layers.Dense(
        units=2,                                                                     # 답변을 하려면 두개의 포지션이 필요(첫부분, 끝부분) unit이 2개이다.
        kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))       

  def call(self, inputs):
    logits = self.final_dense(inputs) # (batch_size, seq_len, 2) -> (4, 384, 2)

    logits = tf.transpose(logits, [2, 0, 1]) # (2, batch_size, seq_len)
    unstacked_logits = tf.unstack(logits, axis=0) # [(batch_size, seq_len), (batch_size, seq_len)] 
    return unstacked_logits[0], unstacked_logits[1] # 답변 시작의 앞부분, 답변 끝부분

In [9]:
class BERTSquad(tf.keras.Model):
    
    def __init__(self,
                 name="bert_squad"):
        super(BERTSquad, self).__init__(name=name)
        
        self.bert_layer = hub.KerasLayer(                                           # call bert layer from web
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        
        self.squad_layer = BertSquadLayer() # final part
    
    def apply_bert(self, inputs):
        # output : 
        _ , sequence_output = self.bert_layer([inputs["input_word_ids"],
                                               inputs["input_mask"],
                                               inputs["input_type_ids"]])
        return sequence_output

    def call(self, inputs):
        seq_output = self.apply_bert(inputs)

        start_logits, end_logits = self.squad_layer(seq_output)
        
        return start_logits, end_logits

In [10]:
bert_layer = hub.KerasLayer(                                           # call bert layer from web
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)



# Training

In [11]:
TRAIN_DATA_SIZE = 88641
NB_BATCHES_TRAIN = 2000 # 시간이 너무오래걸리니 데이터 일부분만 사용하겠다.
BATCH_SIZE = 4
NB_EPOCHS = 3
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [12]:
train_dataset_light = train_dataset.take(NB_BATCHES_TRAIN) # 2000개만 가져온다.

In [13]:
next(iter(train_dataset_light))

({'input_mask': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0],
         [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>,
  'input_type_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>,
  'input_word_ids': <tf.Tensor: shape=(4, 384), dtype=int32, numpy=
  array([[ 101, 2129, 2116, ...,    0,    0,    0],
         [ 101, 2054, 2003, ...,    0,    0,    0],
         [ 101, 2054, 2828, ...,    0,    0,    0],
         [ 101, 2054, 2106, ...,    0,    0,    0]], dtype=int32)>},
 {'end_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 32,  56, 135,  33], dtype=int32)>,
  'start_positions': <tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 29,  52, 133,  30], dtype=int32)>})

In [14]:
bert_squad = BERTSquad()

In [15]:
optimizer = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=NB_BATCHES_TRAIN,
    num_warmup_steps=WARMUP_STEPS)

In [16]:
'''
loss function 틀

def my_loss_function(y_true, y_pred):
return losses
'''


def squad_loss_fn(labels, model_outputs):
    start_positions = labels['start_positions']
    end_positions = labels['end_positions']
    start_logits, end_logits = model_outputs

    start_loss = tf.keras.backend.sparse_categorical_crossentropy(
        start_positions, start_logits, from_logits=True)
    end_loss = tf.keras.backend.sparse_categorical_crossentropy(
        end_positions, end_logits, from_logits=True)
    
    total_loss = (tf.reduce_mean(start_loss) + tf.reduce_mean(end_loss)) / 2

    return total_loss

train_loss = tf.keras.metrics.Mean(name="train_loss")

In [17]:
bert_squad.compile(optimizer,
                   squad_loss_fn)

In [18]:
checkpoint_path = "/content/drive/MyDrive/study/udemy_bert/ckpt_bert_squad/"

ckpt = tf.train.Checkpoint(bert_squad=bert_squad)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [ ]:
for epoch in range(1):
    print("Start of epoch {}".format(epoch+1))
    start = time.time()
    
    train_loss.reset_states()
    
    for (batch, (inputs, targets)) in enumerate(train_dataset_light):
        with tf.GradientTape() as tape: # 자동미분을 기록함
            model_outputs = bert_squad(inputs)
            loss = squad_loss_fn(targets, model_outputs)
        
        gradients = tape.gradient(loss, bert_squad.trainable_variables)
        optimizer.apply_gradients(zip(gradients, bert_squad.trainable_variables))
        
        train_loss(loss)
        
        if batch % 50 == 0:
            ckpt_save_path = ckpt_manager.save()
            print("Epoch {} Batch {} Loss {:.4f}".format(
                epoch+1, batch, train_loss.result()))
        
        #if batch % 500 == 0:
            #ckpt_save_path = ckpt_manager.save()
            #print("Saving checkpoint for epoch {} at {}".format(epoch+1,
            #                                                    ckpt_save_path))
    print("Time taken for 1 epoch: {} secs\n".format(time.time() - start))

In [ ]:
ckpt.restore(ckpt_manager.latest_checkpoint)

# 5. evaluation

## 5-1. Preprocess for evaluation

In [19]:
eval_examples = read_squad_examples(
    "/content/drive/MyDrive/study/udemy_bert/data/dev-v1.1.json",
    is_training=False,
    version_2_with_negative=False)

In [20]:
print(f'question : {eval_examples[0].question_text}')
print(f'document : {eval_examples[0].doc_tokens}')

question : Which NFL team represented the AFC at Super Bowl 50?
document : ['Super', 'Bowl', '50', 'was', 'an', 'American', 'football', 'game', 'to', 'determine', 'the', 'champion', 'of', 'the', 'National', 'Football', 'League', '(NFL)', 'for', 'the', '2015', 'season.', 'The', 'American', 'Football', 'Conference', '(AFC)', 'champion', 'Denver', 'Broncos', 'defeated', 'the', 'National', 'Football', 'Conference', '(NFC)', 'champion', 'Carolina', 'Panthers', '24–10', 'to', 'earn', 'their', 'third', 'Super', 'Bowl', 'title.', 'The', 'game', 'was', 'played', 'on', 'February', '7,', '2016,', 'at', "Levi's", 'Stadium', 'in', 'the', 'San', 'Francisco', 'Bay', 'Area', 'at', 'Santa', 'Clara,', 'California.', 'As', 'this', 'was', 'the', '50th', 'Super', 'Bowl,', 'the', 'league', 'emphasized', 'the', '"golden', 'anniversary"', 'with', 'various', 'gold-themed', 'initiatives,', 'as', 'well', 'as', 'temporarily', 'suspending', 'the', 'tradition', 'of', 'naming', 'each', 'Super', 'Bowl', 'game', 'with

In [21]:
# create tf_record file

eval_writer = FeatureWriter(filename=os.path.join("/content/drive/MyDrive/study/udemy_bert/data/squad/",
                                "eval.tf_record"),
                                is_training=False)

In [22]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)

vocab_file1 = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()   # vocab_file을 다운
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()        # 소문자
tokenizer = FullTokenizer(vocab_file1, do_lower_case)                       # 토크나이저 만들기

In [23]:
do_lower_case

True

In [24]:
def _append_feature(feature, is_padding):
    '''add features to eval_feature list'''
    
    if not is_padding:
        eval_features.append(feature)
    eval_writer.process_feature(feature)

In [25]:
eval_features = []
"""Loads a data file into a list of `InputBatch`s."""
dataset_size = convert_examples_to_features(
    examples=eval_examples, # eval_examples : (question, document)
    tokenizer=tokenizer, # bert_tokenizer
    max_seq_length=384,
    doc_stride=128,
    max_query_length=64,
    is_training=False,
    output_fn=_append_feature, # output_fn 는 feature와 is_padding을 받아야한다.
    batch_size=4)

eval_writer.close()

In [26]:
dataset_size

10836

In [27]:
# load ready to use dataset

BATCH_SIZE = 4

eval_dataset = create_squad_dataset(
    "/content/drive/MyDrive/study/udemy_bert/data/squad/eval.tf_record",
    384,
    BATCH_SIZE,
    is_training=False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: '<' not supported between instances of 'str' and 'Literal'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


# Predictions

In [28]:
RawResult = collections.namedtuple("RawResult",
                                   ["unique_id", "start_logits", "end_logits"])

In [29]:
def get_raw_results(predictions):
    for unique_ids, start_logits, end_logits in zip(predictions['unique_ids'],
                                                    predictions['start_logits'],
                                                    predictions['end_logits']):
        yield RawResult(unique_id=unique_ids.numpy(),
                          start_logits=start_logits.numpy().tolist(),
                          end_logits=end_logits.numpy().tolist())

In [30]:
all_results = []
for count, inputs in enumerate(eval_dataset):
    x, _ = inputs
    unique_ids = x.pop("unique_ids")
    start_logits, end_logits = bert_squad(x, training=False)
    output_dict = dict(
        unique_ids=unique_ids,
        start_logits=start_logits,
        end_logits=end_logits)
    for result in get_raw_results(output_dict):
        all_results.append(result)
    if count % 100 == 0:
        print("{}/{}".format(count, 2709))

0/2709
100/2709
200/2709
300/2709
400/2709
500/2709
600/2709
700/2709
800/2709
900/2709
1000/2709
1100/2709
1200/2709
1300/2709
1400/2709
1500/2709
1600/2709
1700/2709
1800/2709
1900/2709
2000/2709
2100/2709
2200/2709
2300/2709
2400/2709
2500/2709
2600/2709
2700/2709


In [31]:
# Save result
output_prediction_file = "/content/drive/MyDrive/study/udemy_bert/data/squad/predictions.json"
output_nbest_file = "/content/drive/MyDrive/study/udemy_bert/data/squad/nbest_predictions.json"
output_null_log_odds_file = "/content/drive/MyDrive/study/udemy_bert/data/squad/null_odds.json"

write_predictions(
    eval_examples,
    eval_features,
    all_results,
    20,
    30,
    True,
    output_prediction_file,
    output_nbest_file,
    output_null_log_odds_file,
    verbose=False)

# 질문해보자

In [35]:
my_bert_layer = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
    trainable=False)

vocab_file = my_bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = my_bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

In [36]:
def is_whitespace(c):
    '''
    Tell if a chain of characters corresponds to a whitespace or not.
    '''
    if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
        return True
    return False


def whitespace_split(text):
    '''
    Take a text and return a list of "words" by splitting it according to
    whitespaces.
    '''
    doc_tokens = []
    prev_is_whitespace = True
    for c in text:
        if is_whitespace(c):
            prev_is_whitespace = True
        else:
            if prev_is_whitespace:
                doc_tokens.append(c)
            else:
                doc_tokens[-1] += c
            prev_is_whitespace = False
    return doc_tokens

def tokenize_context(text_words):
    '''
    Take a list of words (returned by whitespace_split()) and tokenize each word
    one by one. Also keep track, for each new token, of its original word in the
    text_words parameter.
    '''
    text_tok = []
    tok_to_word_id = []
    for word_id, word in enumerate(text_words):
        word_tok = tokenizer.tokenize(word)
        text_tok += word_tok
        tok_to_word_id += [word_id]*len(word_tok)
    return text_tok, tok_to_word_id


def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)

def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # turns 1 into 0 and vice versa
    return seg_ids


def create_input_dict(question, context):
    '''
    Take a question and a context as strings and return a dictionary with the 3
    elements needed for the model. Also return the context_words, the
    context_tok to context_word ids correspondance and the length of
    question_tok that we will need later.
    '''
    question_tok = tokenizer.tokenize(my_question)

    context_words = whitespace_split(context)
    context_tok, context_tok_to_word_id = tokenize_context(context_words)

    input_tok = question_tok + ["[SEP]"] + context_tok + ["[SEP]"]
    input_tok += ["[PAD]"]*(384-len(input_tok)) # in our case the model has been
                                                # trained to have inputs of length max 384
    input_dict = {}
    input_dict["input_word_ids"] = tf.expand_dims(tf.cast(get_ids(input_tok), tf.int32), 0)
    input_dict["input_mask"] = tf.expand_dims(tf.cast(get_mask(input_tok), tf.int32), 0)
    input_dict["input_type_ids"] = tf.expand_dims(tf.cast(get_segments(input_tok), tf.int32), 0)

    return input_dict, context_words, context_tok_to_word_id, len(question_tok)


In [37]:
doc = '''Neoclassical economics views inequalities in the distribution of income as arising from differences in value added by labor, capital and land. Within labor income distribution is due to differences in value added by different classifications of workers. In this perspective, wages and profits are determined by the marginal value added of each economic actor (worker, capitalist/business owner, landlord). Thus, in a market economy, inequality is a reflection of the productivity gap between highly-paid professions and lower-paid professions.'''

In [38]:
my_question = '''What are examples of economic actors?'''

In [40]:
my_input_dict, my_context_words, context_tok_to_word_id, question_tok_len = create_input_dict(my_question, doc)

# my_input_dict : 3 input data such as input_mask, input_type_ids, input_word_ids
# my_context_words : sep by words
# question_tok_len : [CLS] Questions [SEP] ~~~ 에서 Question의 길이

In [41]:
start_logits, end_logits = bert_squad(my_input_dict, training=False)

In [59]:
# [CLS] Questions [SEP] ~~~ 에서 [SEP]전의것 제거

start_logits_context = start_logits.numpy()[0, question_tok_len+1:]
end_logits_context = end_logits.numpy()[0, question_tok_len+1:]

In [60]:
start_word_id = context_tok_to_word_id[np.argmax(start_logits_context)]
end_word_id = context_tok_to_word_id[np.argmax(end_logits_context)]

In [61]:
# start_word_id가 end_word_id보다 앞서야 한다.
pair_scores = np.ones((len(start_logits_context), len(end_logits_context)))*(-1E10)
for i in range(len(start_logits_context-1)):
    for j in range(i, len(end_logits_context)):
        pair_scores[i, j] = start_logits_context[i] + end_logits_context[j]
pair_scores_argmax = np.argmax(pair_scores)

In [62]:
start_word_id = context_tok_to_word_id[pair_scores_argmax // len(start_logits_context)]
end_word_id = context_tok_to_word_id[pair_scores_argmax % len(end_logits_context)]

In [63]:
predicted_answer = ' '.join(my_context_words[start_word_id:end_word_id+1])
print("The answer to:\n" + my_question + "\nis:\n" + predicted_answer)

The answer to:
What are examples of economic actors?
is:
(worker, capitalist/business owner, landlord).


In [66]:
from IPython.core.display import HTML
display(HTML(f'<h2>{my_question.upper()}</h2>'))
marked_text = str(doc.replace(predicted_answer, f"<mark>{predicted_answer}</mark>"))
display(HTML(f"""<blockquote> {marked_text} </blockquote>"""))